In [10]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import scipy.stats as sp
import matplotlib.pyplot as plt
import json
import logging
import warnings
import os
import subprocess

warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)

logger = logging.getLogger("pymc")
logger.setLevel(logging.ERROR)

# Parameters for the experiment + the varying attribute and the varying values
def run_experiment(
    config_descr,
    number_of_runs,
    varying_attribute,
    varying_values,
    mode_distance,
    std_of_modes,
    weights,
    num_samples,
    num_chains,
    use_init_scheme=False,
    init_scheme= None,
    base_random_seed=None,
    
):
      
    # === Experiment Setup ===
    samples_per_chain = num_samples // num_chains

    # generates pairs of means as symmteric with changed sign (e.g. (1, -1), (2, -2), ...)
    #mean_range = [(i, -i) for i in range(min_mode_value, max_mode_value + 1)]

    # Create configuration folder
    config_folder = f"config_{config_descr}_with_{number_of_runs}"
    os.makedirs(config_folder, exist_ok=True)

    # Save metadata
    metadata_filename = os.path.join(config_folder, f"metadata_config_{config_descr}.json")
    experiment_metadata = {
        "config_descr": config_descr,
        "number_of_runs": number_of_runs,
        "mode_distance": mode_distance,
        "std_of_modes": std_of_modes,
        "num_samples": num_samples,
        "num_chains": num_chains,
        "samples_per_chain": samples_per_chain,
        "weights": weights,
        "init_scheme": init_scheme if use_init_scheme else "None",
        "chain_initialization": init_scheme,
        "base_random_seed": base_random_seed,
        "git_tag": get_git_tag(),
    }
    with open(metadata_filename, "w") as f:
        json.dump(experiment_metadata, f, indent=4)

    # Define fixed colors for each sampler
    sampler_colors = {
        "Metro": "blue",
        "HMC": "red",
        "DEMetro": "green"
    }

    # Initialize random number generator
    rng = np.random.default_rng(base_random_seed) 

    # Precompute IID samples for each mode distance
    iid_samples_dict = {}

    #for means in mean_range:
    #    iid_samples_dict[means] = generate_iid_samples(num_samples=num_samples, means=means, weights=weights, rng=rng)

    # === Run the Experiment ===
    for run_id in range(1, number_of_runs + 1):

        chain_seeds = rng.integers(0, 1_000_000, size=num_chains)

        print(f"\n===== Running Config {config_descr} - Run {run_id} With Chain Seeds : {chain_seeds} =====\n")

        run_folder = os.path.join(config_folder, f"run_{run_id}")
        results_folder = os.path.join(run_folder, "results")
        traces_folder = os.path.join(run_folder, "traces")
        plots_folder = os.path.join(run_folder, "plots")

        os.makedirs(run_folder)
        os.makedirs(results_folder)
        os.makedirs(traces_folder)
        os.makedirs(plots_folder)

        results = []

        for means in mean_range:

            initvals = get_initvals(init_scheme, means, num_chains, rng) if use_init_scheme else None

            # Initialize model
            framework = BimodalPosteriorExample(means=means, weights=weights)
            
            iid_samples = iid_samples_dict[means]
            
            # Run sampling for all samplers
            for sampler_name in ["Metro", "HMC", "DEMetro"]:
                
                print(f"Running for sampler {sampler_name} with means = {means}")

                trace = framework.run_sampling(sampler_name, num_samples=samples_per_chain, num_chains=num_chains, initvals=initvals, use_init_scheme=use_init_scheme, run_random_seed=chain_seeds.tolist())
                
                # Save trace to NetCDF file
                trace_filename = os.path.join(traces_folder, f"{sampler_name}_trace.nc")
                az.to_netcdf(trace, trace_filename)

                # Compute Wasserstein Distance
                ws_distance = sp.wasserstein_distance(trace.posterior["mixed_normal"].values.flatten(), iid_samples)

                trace.posterior["mixed_normal"].values.flatten()

                # Compute R-hat
                r_hat = az.rhat(trace)["mixed_normal"].item()

                # Computes ESS
                ess = az.ess(trace)["mixed_normal"].item()

                results.append({
                    "means": means,
                    "sampler": sampler_name,
                    "wasserstein_distance": ws_distance,
                    "r_hat": r_hat,
                    "ess": ess,
                    "chain_seeds": chain_seeds.tolist(),
                })


        # Convert results to DataFrame
        df_results = pd.DataFrame(results)

        # Extract mode distances (first value of the tuple in "means" column)
        df_results["mode_distance"] = df_results["means"].apply(lambda x: abs(x[0] - x[1]))

        # Sort results
        df_results = df_results.sort_values("mode_distance", ascending=False)

        # initialize plots for all samplers
        fig_ws, ax_ws = plt.subplots(figsize=(10, 6))
        fig_rhat, ax_rhat = plt.subplots(figsize=(10, 6))
        fig_ess, ax_ess = plt.subplots(figsize=(10, 6))

        # Iterate over all samplers and plot the results
        for sampler in df_results["sampler"].unique():
            
            df_sampler = df_results[df_results["sampler"] == sampler]

            # Save CSV files
            csv_filename = os.path.join(results_folder, f"{sampler}_results.csv")
            df_sampler.to_csv(csv_filename, index=False)


            # Plot Wasserstein Distance
            ax_ws.plot(df_sampler["mode_distance"], df_sampler["wasserstein_distance"], 
                marker="o", linestyle="-", label=sampler, 
                color=sampler_colors.get(sampler, "black"))

            # Plot R-hat values
            ax_rhat.plot(df_sampler["mode_distance"], df_sampler["r_hat"], 
                    marker="o", linestyle="-", label=sampler, 
                    color=sampler_colors.get(sampler, "black"))
            
            # Plot ESS values
            ax_ess.plot(df_sampler["mode_distance"], df_sampler["ess"], 
                    marker="o", linestyle="-", label=sampler, 
                    color=sampler_colors.get(sampler, "black"))

        
        # ===== Finalize and Save Wasserstein Plot =====
        ax_ws.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
        ax_ws.set_ylabel("Wasserstein Distance")
        ax_ws.set_title(f"Wasserstein Distance vs Mode Distance for All Samplers (config =_{config_descr})")
        ax_ws.legend(title="Sampler")
        ax_ws.grid(True)
        plot_filename = os.path.join(plots_folder, f"Wasserstein_plot_run_{run_id}.pdf")
        fig_ws.savefig(plot_filename, bbox_inches="tight")
        plt.close(fig_ws)

        # ===== Finalize and Save R-hat Plot =====
        ax_rhat.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
        ax_rhat.set_ylabel("R-hat")
        ax_rhat.set_title(f"R-hat vs Mode Distance for All Samplers (config =_{config_descr})")
        ax_rhat.legend(title="Sampler")
        ax_rhat.grid(True)
        rhat_plot_filename = os.path.join(plots_folder, f"R-hat_plot_run_{run_id}.pdf")
        fig_rhat.savefig(rhat_plot_filename, bbox_inches="tight")
        plt.close(fig_rhat)

        # ===== Finalize and Save ESS Plot =====
        ax_ess.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
        ax_ess.set_ylabel("ESS")
        ax_ess.set_title(f"ESS vs Mode Distance for All Samplers (config =_{config_descr})")
        ax_ess.legend(title="Sampler")
        ax_ess.grid(True)
        ess_plot_filename = os.path.join(plots_folder, f"ESS_plot_run_{run_id}.pdf")
        fig_ess.savefig(ess_plot_filename, bbox_inches="tight")
        plt.close(fig_ess)


    print("\n===== All Runs Completed Successfully! =====\n")

    # ===== GLOBAL RESULTS FOLDER =====
    global_folder = os.path.join(config_folder, "global_results")
    global_results_folder = os.path.join(global_folder, "results")
    global_plots_folder = os.path.join(global_folder, "plots")

    os.makedirs(global_folder)
    os.makedirs(global_results_folder)
    os.makedirs(global_plots_folder)

    # Collect all results from all runs
    df_all_runs = []

    for run_id in range(1, number_of_runs + 1):
        run_folder = os.path.join(config_folder, f"run_{run_id}")
        results_folder = os.path.join(run_folder, "results")

        for sampler in ["Metro", "HMC", "DEMetro"]:
            csv_filename = os.path.join(results_folder, f"{sampler}_results.csv")
            df_run = pd.read_csv(csv_filename)
            df_run["run_id"] = run_id 
            df_run["sampler"] = sampler  
            df_all_runs.append(df_run)


    # Combine all results into a single data frame 
    df_all_runs = pd.concat(df_all_runs, ignore_index=True)

    # Ensure 'means' are stored as tuples
    df_all_runs["means"] = df_all_runs["means"].apply(eval)

    # Initialize global plots
    fig_ws, ax_ws = plt.subplots(figsize=(10, 6))
    fig_rhat, ax_rhat = plt.subplots(figsize=(10, 6))
    fig_ess, ax_ess = plt.subplots(figsize=(10, 6))

    for sampler in ["Metro", "HMC", "DEMetro"]:
        df_sampler = df_all_runs[df_all_runs["sampler"] == sampler]

        # Pivot tables: rows = mode_distance, columns = run_id, values = metrics
        df_ws = df_sampler.pivot_table(index="mode_distance", columns="run_id", values="wasserstein_distance")
        df_rhat = df_sampler.pivot_table(index="mode_distance", columns="run_id", values="r_hat")
        df_ess = df_sampler.pivot_table(index="mode_distance", columns="run_id", values="ess")

        # Compute mean and standard deviation for error bars
        ws_mean, ws_std = df_ws.mean(axis=1), df_ws.std(axis=1)
        rhat_mean, rhat_std = df_rhat.mean(axis=1), df_rhat.std(axis=1)
        ess_mean, ess_std = df_ess.mean(axis=1), df_ess.std(axis=1)

        color = sampler_colors.get(sampler, "black")

        # Plot with error bars
        ax_ws.errorbar(ws_mean.index, ws_mean, yerr=ws_std, fmt="o-", label=sampler, color=color, capsize=5)
        ax_rhat.errorbar(rhat_mean.index, rhat_mean, yerr=rhat_std, fmt="o-", label=sampler, color=color, capsize=5)
        ax_ess.errorbar(ess_mean.index, ess_mean, yerr=ess_std, fmt="o-", label=sampler, color=color, capsize=5)

        # Save global averages 
        df_global_avg = pd.DataFrame({
            "mode_distance": ws_mean.index,
            "global_avg_ws": ws_mean.values,
            "global_avg_ws_std": ws_std.values,
            "global_avg_rhat": rhat_mean.values,
            "global_avg_rhat_std": rhat_std.values,
            "global_avg_ess": ess_mean.values,
            "global_avg_ess_std": ess_std.values,
        })

        sampler_csv_filename = os.path.join(global_results_folder, f"Global_results_{sampler}.csv")
        df_global_avg.to_csv(sampler_csv_filename, index=False)

    # ===== Save Global Wasserstein Plot =====
    ax_ws.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
    ax_ws.set_ylabel("Average Wasserstein Distance")
    ax_ws.set_title(f"Avrg Wasserstein Distance ({number_of_runs} Runs, config = {config_descr})")
    ax_ws.legend(title="Sampler")
    ax_ws.grid(True)
    fig_ws.savefig(os.path.join(global_plots_folder, "Wasserstein_global_plot.pdf"), bbox_inches="tight")
    plt.close(fig_ws)

    # ===== Save Global R-hat Plot =====
    ax_rhat.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
    ax_rhat.set_ylabel("Average R-hat")
    ax_rhat.set_title(f"Avrg R-hat Values ({number_of_runs} Runs, config = {config_descr})")
    ax_rhat.legend(title="Sampler")
    ax_rhat.grid(True)
    fig_rhat.savefig(os.path.join(global_plots_folder, "Rhat_global_plot.pdf"), bbox_inches="tight")
    plt.close(fig_rhat)

    # ===== Save Global ESS Plot =====
    ax_ess.set_xlabel("Mode Distance (|Mean1 - Mean2|)")
    ax_ess.set_ylabel("Average ESS")
    ax_ess.set_title(f"Avrg ESS ({number_of_runs} Runs,  config = {config_descr})")
    ax_ess.legend(title="Sampler")
    ax_ess.grid(True)
    fig_ess.savefig(os.path.join(global_plots_folder, "ESS_global_plot.pdf"), bbox_inches="tight")
    plt.close(fig_ess)

    print(f"\n===== Config {config_descr} Completed Successfully! =====\n")


class BimodalPosteriorExample:
    
    def __init__(self, means, weights=[1,1]):
        self.model = self._bimodal_posterior(weights, means)
    
    def _bimodal_posterior(self, weights, means):        
        with pm.Model() as model:
            
            # Mixture weights (equal probability for both components)
            w = pm.Dirichlet("w", a=np.array(weights))

            # Component distributions
            mu = pm.Normal.dist(means, 1)
            
            # Mixture distribution
            mixed_normal = pm.Mixture("mixed_normal", w=w, comp_dists=mu)

        return model

    def run_sampling(self, 
                     sampler_name, 
                     num_samples=2000, 
                     tune=1000, 
                     num_chains=2, 
                     initvals=None,
                     use_init_scheme=False,
                     run_random_seed= None
                     ):
    

        with self.model:

            # Define which sampler to use
            if sampler_name == "Metro":
                sampler = pm.Metropolis()
            elif sampler_name == "HMC":
                sampler = pm.NUTS()
            elif sampler_name == "DEMetro":
                sampler = pm.DEMetropolis()
            else:
                raise ValueError(f"Unknown sampler: {sampler_name}")

            if use_init_scheme:
                trace = pm.sample(num_samples, tune=tune, step=sampler, initvals= initvals,chains=num_chains, return_inferencedata=True, progressbar=False, random_seed= run_random_seed)
            else:
                trace = pm.sample(num_samples, tune=tune, step=sampler, chains=num_chains, return_inferencedata=True, progressbar=False, random_seed= run_random_seed)
        
        return trace
    


def generate_iid_samples(means, num_samples=2000, weights=[1,1], rng= None):
    """
    Generate IID samples from the bimodal posterior directly.
    """

    rng = rng or np.random.default_rng()
    # Normalize weights
    weights = np.array(weights) / np.sum(weights)
    modes = rng.choice(means, size=num_samples, p=weights)
    # Sample from N(mode, 1)
    iid_samples = rng.normal(loc=modes, scale=1, size=num_samples)

    return iid_samples



def get_initvals(init_scheme, means, num_chains, rng=None):
    """Generates initialization values based on the chosen scheme."""

    middle_point = sum(means) / 2

    if init_scheme == "half_per_mode":
        # Half the chains start near the first mode, half near the second mode
        initvals = [
            {"w": np.array([0.9, 0.1]), "mixed_normal": means[0]} for _ in range(num_chains // 2)
        ] + [
            {"w": np.array([0.1, 0.9]), "mixed_normal": means[1]} for _ in range(num_chains // 2)
        ]

    elif init_scheme == "all_in_middle":
        # All chains start in the middle between the two modes
        initvals = [{"w": np.array([0.5, 0.5]), "mixed_normal": middle_point} for _ in range(num_chains)]

    elif init_scheme == "random":
        # Chains are initialized randomly between the modes
        initvals = [
            {"w": rng.dirichlet(np.ones(2)), "mixed_normal": rng.uniform(means[0], means[1])}
            for _ in range(num_chains)
        ]

    elif init_scheme == "all_near_first_mode":
        # All chains start near the first mode
        initvals = [{"w": np.array([0.9, 0.1]), "mixed_normal": means[0]} for _ in range(num_chains)]

    elif init_scheme == "all_near_second_mode":
        # All chains start near the second mode
        initvals = [{"w": np.array([0.1, 0.9]), "mixed_normal": means[1]} for _ in range(num_chains)]

    else:
        raise ValueError(f"Unknown initialization scheme: {init_scheme}")

    return initvals

# Function to get the current git tag
def get_git_tag():
        try:
            tag = subprocess.check_output(["git", "describe", "--tags"], stderr=subprocess.DEVNULL).strip().decode()
            return tag
        except subprocess.CalledProcessError:
            return "No tag found"

In [9]:
configurations_experiment_1 = [
    
    # optimal DEMetropolis := unequal mode wieghts and custom chain initialization
    {
        "config_descr": "Optimal DEMetropolis",
        "runs_max": 2,
        "min_mode_value": 5,
        "max_mode_value": 5,
        "std_of_modes": 1,
        "weights": [1, 6],
        "num_samples": 1000,
        "num_chains": 4,
        "use_custom_chain_init": True,
        "base_random_seed": 42
    },

    {
        "config_descr": "Only Unequal weights",
        "runs_max": 2,
        "min_mode_value": 5,
        "max_mode_value": 5,
        "std_of_modes": 1,
        "weights": [1, 6],
        "num_samples": 1000,
        "num_chains": 4,
        "use_custom_chain_init": False,
        "base_random_seed": 42
    },
    
    {
        "config_descr": "Only Init Scheme",
        "runs_max": 2,
        "min_mode_value": 5,
        "max_mode_value": 5,
        "std_of_modes": 1,
        "weights": [1, 1],
        "num_samples": 1000,
        "num_chains": 4,
        "use_custom_chain_init": True,
        "base_random_seed": 42
    },

    # fair case := equal mode weights and no custom chain initialization
    {
        "config_descr": "Fair Case",
        "runs_max": 2,
        "min_mode_value": 5,
        "max_mode_value": 5,
        "std_of_modes": 1,
        "weights": [1, 1],
        "num_samples": 1000,
        "num_chains": 4,
        "use_custom_chain_init": False,
        "base_random_seed": 42
    },
]


configurations_experiment_temp = [
    {
        "config_descr": "Test",
        "runs_max": 3,
        "min_mode_value": 9,
        "max_mode_value": 10,
        "std_of_modes": 1,
        "weights": [1, 6],
        "num_samples": 100,
        "num_chains": 4,
        "use_custom_chain_init": True,
        "base_random_seed": 42
    },
        {
        "config_descr": "Test 2",
        "runs_max": 3,
        "min_mode_value": 9,
        "max_mode_value": 10,
        "std_of_modes": 1,
        "weights": [1, 1],
        "num_samples": 100,
        "num_chains": 4,
        "use_custom_chain_init": False,
        "base_random_seed": 42
    },
]

for config in configurations_experiment_temp:
    run_experiment(
        config_descr=config["config_descr"],
        runs_max=config["runs_max"],
        min_mode_value=config["min_mode_value"],
        max_mode_value=config["max_mode_value"],
        std_of_modes=config["std_of_modes"],
        weights=config["weights"],
        num_samples=config["num_samples"],
        num_chains=config["num_chains"],
        use_custom_chain_init=config["use_custom_chain_init"],
    )



===== Running Config Test - Run 1 With Chain Seeds : [372142 819662 426875  77414] =====

Running for sampler Metro with means = (9, -9)
Running for sampler HMC with means = (9, -9)
Running for sampler DEMetro with means = (9, -9)
Running for sampler Metro with means = (10, -10)
Running for sampler HMC with means = (10, -10)
Running for sampler DEMetro with means = (10, -10)

===== Running Config Test - Run 2 With Chain Seeds : [958719 260518 898061  49324] =====

Running for sampler Metro with means = (9, -9)
Running for sampler HMC with means = (9, -9)
Running for sampler DEMetro with means = (9, -9)
Running for sampler Metro with means = (10, -10)
Running for sampler HMC with means = (10, -10)
Running for sampler DEMetro with means = (10, -10)

===== Running Config Test - Run 3 With Chain Seeds : [485784 805595  45231 365171] =====

Running for sampler Metro with means = (9, -9)
Running for sampler HMC with means = (9, -9)
Running for sampler DEMetro with means = (9, -9)
Running fo